# installation dependances

In [1]:
import os

print(os.environ.keys())

if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

KeysView(environ({'SHELL': '/bin/bash', 'NV_LIBCUBLAS_VERSION': '12.4.5.8-1', 'NVIDIA_VISIBLE_DEVICES': 'void', 'KUBERNETES_SERVICE_PORT_HTTPS': '443', 'NV_NVML_DEV_VERSION': '12.4.127-1', 'NV_CUDNN_PACKAGE_NAME': 'libcudnn9-cuda-12', 'KUBERNETES_SERVICE_PORT': '443', 'AWS_S3_ENDPOINT': 'minio.lab.sspcloud.fr', 'WORKSPACE_DIR': '/home/onyxia/work', 'NV_LIBNCCL_DEV_PACKAGE': 'libnccl-dev=2.21.5-1+cuda12.4', 'NV_LIBNCCL_DEV_PACKAGE_VERSION': '2.21.5-1', 'HOSTNAME': 'jupyter-python-gpu-750753-0', 'PYTHON_VERSION': '3.12.9', 'GIT_USER_MAIL': 'adrien.quentin@insee.fr', 'DARK_MODE': 'true', 'NVIDIA_REQUIRE_CUDA': 'cuda>=12.4 brand=tesla,driver>=470,driver<471 brand=unknown,driver>=470,driver<471 brand=nvidia,driver>=470,driver<471 brand=nvidiartx,driver>=470,driver<471 brand=geforce,driver>=470,driver<471 brand=geforcertx,driver>=470,driver<471 brand=quadro,driver>=470,driver<471 brand=quadrortx,driver>=470,driver<471 brand=titan,driver>=470,driver<471 brand=titanrtx,driver>=470,driver<471 b

# configuration du model

In [2]:
from unsloth import FastModel
import torch
max_seq_length = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-pt-unsloth-bnb-4bit",
    max_seq_length = max_seq_length, # Choose any for long context!
    load_in_4bit = False,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-03 12:50:12 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0. vLLM: 0.8.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.568 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


In [3]:

model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = False,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
from datasets import load_dataset

# Load CSV into Hugging Face dataset
dataset = load_dataset("csv", data_files="fr-train-dataset2.csv")

# Access the split (train by default)
train_dataset = dataset["train"]

# Preview
print(train_dataset[0])
train_dataset

{'Address': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'Building_Number': 147, 'City': 'Paris', 'Recipient': 'Supermarché Carrefour', 'Street_Name_old': 'Rue de Rivoli', 'Street_Name': 'de Rivoli', 'type_voie': 'Rue', 'Zip_Code': 75001, 'Country': 'France', 'repetition': None}


Dataset({
    features: ['Address', 'Building_Number', 'City', 'Recipient', 'Street_Name_old', 'Street_Name', 'type_voie', 'Zip_Code', 'Country', 'repetition'],
    num_rows: 2695
})

In [5]:
train_dataset[0]["Address"]

'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France'

In [6]:
# Define columns to exclude from tgt
exclude_cols = ["Address", "Street_Name_old"]  # 👈 Add any column to ignore here

# Determine tgt columns
tgt_cols = [col for col in train_dataset.column_names if col not in exclude_cols]
src_col = "Address"


# Transform
def transform(example):
    return {
        "question": example[src_col],
        "answer": {col: example[col] for col in tgt_cols}
    }

dataset_refined = train_dataset.map(transform)


# Preview
print(dataset_refined[0])


print(dataset_refined.column_names)


{'Address': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'Building_Number': 147, 'City': 'Paris', 'Recipient': 'Supermarché Carrefour', 'Street_Name_old': 'Rue de Rivoli', 'Street_Name': 'de Rivoli', 'type_voie': 'Rue', 'Zip_Code': 75001, 'Country': 'France', 'repetition': None, 'question': 'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France', 'answer': {'Building_Number': 147, 'City': 'Paris', 'Country': 'France', 'Recipient': 'Supermarché Carrefour', 'Street_Name': 'de Rivoli', 'Zip_Code': 75001, 'repetition': None, 'type_voie': 'Rue'}}
['Address', 'Building_Number', 'City', 'Recipient', 'Street_Name_old', 'Street_Name', 'type_voie', 'Zip_Code', 'Country', 'repetition', 'question', 'answer']


In [7]:
dataset_refined[0]["answer"]

{'Building_Number': 147,
 'City': 'Paris',
 'Country': 'France',
 'Recipient': 'Supermarché Carrefour',
 'Street_Name': 'de Rivoli',
 'Zip_Code': 75001,
 'repetition': None,
 'type_voie': 'Rue'}

In [8]:
dataset_refined[0]["question"]

'Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France'

# configuration de l'entrainement

In [9]:
import json

def format_example(example):
    return {
        "text": f"Adresse : {example['question']}\nChamps : {json.dumps(example['answer'], ensure_ascii=False)}"
    }

formatted_dataset = dataset_refined.map(format_example)
formatted_dataset = formatted_dataset.remove_columns(
    [col for col in formatted_dataset.column_names if col != "text"]
)

print(formatted_dataset[0])



{'text': 'Adresse : Supermarché Carrefour, 147 Rue de Rivoli, 75001 Paris, France\nChamps : {"Building_Number": 147, "City": "Paris", "Country": "France", "Recipient": "Supermarché Carrefour", "Street_Name": "de Rivoli", "Zip_Code": 75001, "repetition": null, "type_voie": "Rue"}'}


In [11]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    args=SFTConfig(
        max_seq_length=1024,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=30,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        report_to="none",
    ),
)


Unsloth: Switching to float32 training since model cannot work with float16


In [12]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos>Adresse : 79 Place de la République, 35000 Rennes, France\nChamps : {"Building_Number": 79, "City": "Rennes", "Country": "France", "Recipient": null, "Street_Name": "de la République", "Zip_Code": 35000, "repetition": null, "type_voie": "Place"}'

In [13]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,695 | Num Epochs = 1 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 5,031,936/1,004,917,888 (0.50% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
1,2.474700
2,2.458400
3,2.415100
4,2.398700
5,2.348100
6,2.234600
7,2.031600
8,1.872700
9,1.749100
10,1.578200


In [17]:
#trainer.save_model("gemma3-address-parser")
model.save_pretrained("gemma3-address-parser") 
tokenizer.save_pretrained("gemma3-address-parser")
model.config.save_pretrained("gemma3-address-parser")

model.save_pretrained_gguf(
        "gemma3-address-parser",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Unsloth: Updating system package directories
Unsloth: All commands will now use admin permissions (sudo)
Unsloth: Install GGUF and other packages
Unsloth GGUF:hf-to-gguf:Loading model: gemma3-address-parser
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting special token type bos to 2
Unsloth GGUF:gguf.vocab:Setting special token type eos to 1
Unsloth GGUF:gguf.vocab:Setting special token type unk to 3
Unsloth GGUF:gguf.vocab:Setting special token type pad to 0
Unsloth GGUF:gguf.vocab:Setting add_bos_token to True
Unsloth GGUF:gguf.vocab:Setting add_eos_token to False
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:gguf.gguf_writer:Writing the following files:
Unsloth GGUF:gguf.gguf_writer:gemma3-address-parser.Q8_0.gguf: n_tensors = 0, tota

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Writing: 0.00byte [00:00, ?byte/s]
Writing: 0.00byte [00:00, ?byte/s]
Unsloth GGUF:hf-to-gguf:Model successfully exported to ./
Unsloth GGUF:hf-to-gguf:Loading model: gemma3-address-parser
Unsloth GGUF:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
Unsloth GGUF:hf-to-gguf:Exporting model...
Unsloth GGUF:hf-to-gguf:Set meta model
Unsloth GGUF:hf-to-gguf:Set model parameters
Unsloth GGUF:hf-to-gguf:Set model tokenizer
Unsloth GGUF:gguf.vocab:Setting special token type bos to 2
Unsloth GGUF:gguf.vocab:Setting special token type eos to 1
Unsloth GGUF:gguf.vocab:Setting special token type unk to 3
Unsloth GGUF:gguf.vocab:Setting special token type pad to 0
Unsloth GGUF:gguf.vocab:Setting add_bos_token to True
Unsloth GGUF:gguf.vocab:Setting add_eos_token to False
Unsloth GGUF:hf-to-gguf:Set model quantization version
Unsloth GGUF:gguf.gguf_writer:Writing the following files:
Unsloth GGUF:gguf.gguf_writer:gemma3-address-parser.Q8_0.gguf: n_tensors = 0, total_size = negligibl

Unsloth: GGUF conversion:   0%|          | 0/100 [00:00<?, ?it/s]

Writing: 0.00byte [00:00, ?byte/s]
Writing: 0.00byte [00:00, ?byte/s]
Unsloth GGUF:hf-to-gguf:Model successfully exported to ./


RuntimeError: Unsloth: Failed to convert llama.cpp/unsloth_convert_hf_to_gguf.py to GGUF.

# run it

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("gemma3-address-parser", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("gemma3-address-parser", torch_dtype=torch.float16).cuda()


In [ ]:
address = "Leclerc, 10 avenue Victor Hugo, 92200 Neuilly-sur-Seine"

prompt = f"Adresse : {address}\nChamps :"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=64,
    temperature = 1.0, top_p = 0.95, top_k = 64,
)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))
